In [ ]:
# default_exp a3c.a3c_data
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
from nbdev.export2html import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# A3C Data

> A decoupled actor critic agent which trains on data collected from environments running in a completely separate process.

In [ ]:
# export
# from fastai.basic_data import *
from fastai.torch_core import *
from fastai.callbacks import *
from fastrl.wrappers import *
from fastrl.basic_agents import *
from fastrl.basic_train import *
from fastrl.data_block import *
from fastrl.metrics import *
from fastai.basic_train import *
from dataclasses import asdict
from functools import partial
from fastprogress.fastprogress import IN_NOTEBOOK
from fastcore.utils import *
import torch.multiprocessing as mp
from queue import Empty
import textwrap
import logging
import gym

logging.basicConfig(format='[%(asctime)s] p%(process)s line:%(lineno)d %(levelname)s - %(message)s',
                    datefmt='%m-%d %H:%M:%S')
_logger=logging.getLogger(__name__)

In [ ]:
# hide
_logger.setLevel('INFO')
from fastcore.foundation import *
import sys

In [ ]:
# export
@safe_fit
def a3c_data_fitter(model,agent,ds,data_queue,pause_event,
                    cancel_event,metric_queue):
    dataset=ds()
    while not cancel_event.is_set():
        for xb,yb in dataset:
            data_queue.put(yb)
            if pause_event.is_set():cancel_event.wait(0.1)
            if cancel_event.is_set():break
        if cancel_event.is_set():break
        if metric_queue is not None:
            rs=dataset.pop_total_r()
            if len(rs)!=0:metric_queue.put(TotalRewards(np.mean(rs)))

@dataclass
class A3CLearner(AgentLearner):
    fitter:Callable=a3c_data_fitter
    batch_sz:int=100
        
    def __post_init__(self):
        super(A3CLearner,self).__post_init__()
        if self.model is None:self.model=self.agent.model
        if self.agent.model is None: self.agent.model=self.model
        self.model.share_memory()
        
    def predict(self,s):
        out=self.model(s)
        if type(out)==tuple:return out[0]
        return out

In [ ]:
batch=[
 Experience(s=tensor([[-0.0285,  0.1640, -0.0033, -0.3421]]),sp=tensor([[-0.0285,  0.1640, -0.0033, -0.3421]]),
            a=tensor([1]),r=tensor([1.]),d=tensor([0.]),agent_s=tensor([[[0.]]])),
 Experience(s=tensor([[-0.0252, -0.0311, -0.0101, -0.0504]]),sp=tensor([[-0.0252, -0.0311, -0.0101, -0.0504]]),
            a=tensor([0]),r=tensor([1.]),d=tensor([0.]),agent_s=tensor([[[0.]]])),
 Experience(s=tensor([[-0.0258, -0.2261, -0.0111,  0.2391]]),sp=tensor([[-0.0258, -0.2261, -0.0111,  0.2391]]),
            a=tensor([0]),r=tensor([1.]),d=tensor([0.]),agent_s=tensor([[[0.]]])),
 Experience(s=tensor([[-0.0517, -0.2260,  0.0195,  0.2377]]),sp=tensor([[-0.0517, -0.2260,  0.0195,  0.2377]]),
            a=tensor([1]),r=tensor([1.]),d=tensor([0.]),agent_s=tensor([[[0.]]])),
 Experience(s=tensor([[-0.0562, -0.4214,  0.0242,  0.5365]]),sp=tensor([[-0.0562, -0.4214,  0.0242,  0.5365]]),
            a=tensor([0]),r=tensor([1.]),d=tensor([0.]),agent_s=tensor([[[0.]]])),
 Experience(s=tensor([[-0.0647, -0.6169,  0.0349,  0.8367]]),sp=tensor([[-0.0647, -0.6169,  0.0349,  0.8367]]),
            a=tensor([0]),r=tensor([1.]),d=tensor([1.]),agent_s=tensor([[[0.]]]))
]
class LinearA2C(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(LinearA2C, self).__init__()

        self.policy = nn.Sequential(
            nn.Linear(input_shape[0], 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

        self.value = nn.Sequential(
            nn.Linear(input_shape[0], 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def _get_conv_out(self, shape):
        o=self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        fx=x.float()
        return self.policy(fx),self.value(fx)
model=LinearA2C((4,),2)

In [ ]:
def r_estimate(s,r,d_mask,model,val_gamma,device):
    "Returns rewards `r` estimated direction by `model` from states `s`"
    r_np=np.array(r,dtype=np.float32)
    if d_mask:
        s_v=torch.FloatTensor(s).to(device)
        v=model(s_v)[1] # Remember that models are going to return the actions and the values
        v_np=v.data.cpu().numpy()[:,0]
        r_np[d_mask]+=val_gamma*v_np
    return r_np

def unbatch(batch,model,last_val_gamma,device='cpu')->Tuple(List,List,List):
    s,a,r,d_mask,sp=[],[],[],[],[]
    for i,exp in enumerate(batch):
        s.append(exp.s.numpy())
        a.append(int(exp.a.numpy())) # TODO can we change this to toggle between discrete and continuous actions?
        r.append(exp.r.numpy().astype(np.float32))
        if int(exp.d)==0:
            d_mask.append(i)
            sp.append(exp.sp.numpy())
    s_t=torch.FloatTensor(s).to(device)
    a_t=torch.LongTensor(a).to(device)
    
    r_np=r_estimate(sp,r,d_mask,model,last_val_gamma,device)
    estimated_r=torch.FloatTensor(r_np).to(device)
    return s_t,a_t,estimated_r

In [ ]:
unbatch(batch,model,2)

(tensor([[[-0.0285,  0.1640, -0.0033, -0.3421]],
 
         [[-0.0252, -0.0311, -0.0101, -0.0504]],
 
         [[-0.0258, -0.2261, -0.0111,  0.2391]],
 
         [[-0.0517, -0.2260,  0.0195,  0.2377]],
 
         [[-0.0562, -0.4214,  0.0242,  0.5365]],
 
         [[-0.0647, -0.6169,  0.0349,  0.8367]]]),
 tensor([1, 0, 0, 1, 0, 0]),
 tensor([[1.2883],
         [1.2978],
         [1.3391],
         [1.3321],
         [1.3981],
         [1.0000]]))

In [ ]:
# export
class A3CTrainer(LearnerCallback):
    def __init__(self,*args,**kwargs):
        super(A3CTrainer,self).__init__(*args,**kwargs)
        self.batch=[]
    
    def on_train_begin(self,**kwargs):
        self.batch.clear()
    
    def on_batch_begin(self,last_target,**kwargs):
        self.batch.extend([Experience(**o) for o in last_target])
        if len(self.batch)<self.learn.batch_sz:return
        
    def on_backward_begin(self,*args,**kwargs): return {'skip_bwd':True,'skip_validate':True}
    

In [ ]:
import pytest
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',display=False,firstlast=True,add_valid=False,max_steps=100,n_processes=1,n_envs=1)
model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
agent=PolicyAgent(model=model)
learn=A3CLearner(data,model,agent=agent,callback_fns=[A3CTrainer,RewardMetric],loss_func=fake_loss)
learn.fit(3,lr=0.01,wd=1)

In [ ]:
data=AsyncExperienceSourceDataBunch.from_env('CartPole-v1',display=False,firstlast=False,add_valid=False,max_steps=50,n_processes=1,n_envs=1)
model=nn.Sequential(nn.Linear(4,5),nn.ReLU(),nn.Linear(5,2))
agent=PolicyAgent(model=model)
learn=A3CLearner(data,model,agent=agent,callback_fns=[A3CTrainer,RewardMetric],loss_func=fake_loss)
learn.fit(10,lr=0.01,wd=1)

In [ ]:
# hide
from nbdev.export import *
notebook2script()
notebook2html(n_workers=0)

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 02_callbacks.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_metrics.ipynb.
Converted 05_data_block.ipynb.
Converted 06_basic_train.ipynb.
Converted 12_a3c.a3c_data.ipynb.
Converted index.ipynb.
Converted notes.ipynb.
converting: /opt/project/fastrl/nbs/12_a3c.a3c_data.ipynb


/opt/conda/envs/fastrl/lib/python3.7/site-packages/jupyter_client/manager.py:358: FutureWarning: Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).
  FutureWarning)


converting: /opt/project/fastrl/nbs/05_data_block.ipynb
